## Downloading the data

Using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

In [2]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2019-05-12 09:38:22--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  47.6MB/s    in 1.7s    

2019-05-12 09:38:24 (47.6 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Preparing and Processing the data

In [3]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [7]:
print(train_X[100])
print(train_y[100])

Ohhh man! Now this is what I'm talking about! As far as bad/cheesy horror flicks go this movie was truly in a class of its own. A real gem!<br /><br />First off, the film wasn't originally in English. That's okay because the voice dubbing was truly exceptional! Here is my favorite excerpt from the dialog (and there is plenty more where this came from) "I'm feeling a little better. I'm just thirsty FOR YOUR BLOOD!"<br /><br />And what drama! Here is a play by play recap of the interaction between the military and scientists<br /><br />Scene 1 Scientist: "You can't do that It'll be a disaster!" -- Military Officer: "That's just science fiction" (he then proceeds to cause a complete disaster just like the scientist predicted).<br /><br />Scene 2 Scientist: "If you do that many people will die!!!" -- Military Officer: "you don't know what you're talking about." (he does it and many people die).<br /><br />Scene 3 Scientist: "Don't do that It'll kill everyone!" -- Military Officer: 

### Tokenization

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [9]:
words = review_to_words(train_X[100])
words

['ohhh',
 'man',
 'talk',
 'far',
 'bad',
 'cheesi',
 'horror',
 'flick',
 'go',
 'movi',
 'truli',
 'class',
 'real',
 'gem',
 'first',
 'film',
 'origin',
 'english',
 'okay',
 'voic',
 'dub',
 'truli',
 'except',
 'favorit',
 'excerpt',
 'dialog',
 'plenti',
 'came',
 'feel',
 'littl',
 'better',
 'thirsti',
 'blood',
 'drama',
 'play',
 'play',
 'recap',
 'interact',
 'militari',
 'scientist',
 'scene',
 '1',
 'scientist',
 'disast',
 'militari',
 'offic',
 'scienc',
 'fiction',
 'proce',
 'caus',
 'complet',
 'disast',
 'like',
 'scientist',
 'predict',
 'scene',
 '2',
 'scientist',
 'mani',
 'peopl',
 'die',
 'militari',
 'offic',
 'know',
 'talk',
 'mani',
 'peopl',
 'die',
 'scene',
 '3',
 'scientist',
 'kill',
 'everyon',
 'militari',
 'offic',
 'nonsens',
 'proce',
 'kill',
 'everyon',
 'scene',
 '4',
 '5',
 '6',
 '7',
 'get',
 'idea',
 'enough',
 'scene',
 'realli',
 'stood',
 'instant',
 'classic',
 'one',
 'scene',
 'militari',
 'liter',
 '10',
 'guy',
 'point',
 'gun',
 '

In [10]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


### Creating a word dictionary

In [12]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
   
   word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    
    for words in data:
        for single_word in words:
            if single_word in word_count:
                word_count[single_word] += 1
            else:
                word_count[single_word] = 1    

    sorted_words = [item[0] for item in sorted(word_count.items(), key=lambda x: x[1], reverse=True)]
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [13]:
word_dict = build_dict(train_X)

In [15]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [16]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

In [17]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [18]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [19]:
train_X[0]

array([   5,  841,    3, 1942, 1491,    1,    1,  939,  951,  476, 3376,
          1,   14,    1,  514, 3486,    4,  514, 1758, 1029,  269,    4,
         36,   56,   11,    1,  362,  951,    1,  498, 1663, 2967,    1,
        172,   51,  932,   31,  613,  139, 1007, 4284,  271,  172,  482,
       4057, 1576,  172, 3816,   14, 1647,  137,    1, 4725,  232,    1,
       1380,  476, 2617,    3,  270,  833,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Upload the data to S3

In [20]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Uploading the training data to the SageMaker default S3 bucket so can provide access to it while training model.

In [21]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [22]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Build and Train LSTM Model

In [23]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

In [24]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [25]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            output = model.forward(batch_X)
            
            loss = loss_fn(output, batch_y)
            loss.backward()
            
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [26]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6934450149536133
Epoch: 2, BCELoss: 0.6832457780838013
Epoch: 3, BCELoss: 0.6738565683364868
Epoch: 4, BCELoss: 0.6631478071212769
Epoch: 5, BCELoss: 0.649707293510437


### Training the model

In [27]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [28]:
estimator.fit({'training': input_data})

2019-05-12 09:39:55 Starting - Starting the training job...
2019-05-12 09:39:57 Starting - Launching requested ML instances......
2019-05-12 09:40:59 Starting - Preparing the instances for training......
2019-05-12 09:42:10 Downloading - Downloading input data
2019-05-12 09:42:10 Training - Downloading the training image......
2019-05-12 09:43:12 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-05-12 09:43:12,935 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-05-12 09:43:12,959 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-05-12 09:43:15,967 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-05-12 09:43:16,499 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-05-12 09:43:16,499 sagemak

Model loaded with embedding_dim 32, hidden_dim 200, vocab_size 5000.
Epoch: 1, BCELoss: 0.6731544465434794
Epoch: 2, BCELoss: 0.6056633664637195
Epoch: 3, BCELoss: 0.53288278470234
Epoch: 4, BCELoss: 0.4649423561534103
Epoch: 5, BCELoss: 0.4212734291748125
Epoch: 6, BCELoss: 0.37968380840457217
Epoch: 7, BCELoss: 0.348031721553024
Epoch: 8, BCELoss: 0.3338224018106655
Epoch: 9, BCELoss: 0.299709001365973
Epoch: 10, BCELoss: 0.27468530468794766
2019-05-12 09:46:28,330 sagemaker-containers INFO     Reporting training SUCCESS

2019-05-12 09:46:35 Uploading - Uploading generated training model
2019-05-12 09:46:35 Completed - Training job completed
Billable seconds: 273


## Deploying the model for testing

In [29]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------------------------------------------------------------------------!

In [30]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [31]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [32]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8552

In [34]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [35]:
test_data = [np.array(convert_and_pad(word_dict, review_to_words(test_review))[0])]

In [36]:
predictor.predict(test_data)

array(0.65504014, dtype=float32)

In [37]:
estimator.delete_endpoint()

## Deploying the model

In [39]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

----------------------------------------------------------------------------------------------------!

In [43]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                rlt = predictor.predict(review_input)
                rtn = 1 if(b'1.0' == rlt) else 0
                results.append(rtn)
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [44]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.846

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [46]:
predictor.predict(test_review)

b'1.0'

In [48]:
predictor.endpoint

'sagemaker-pytorch-2019-05-12-09-57-50-885'

In [50]:
predictor.delete_endpoint()